In [1]:
import os
from DSSCircuit_Interface import * 
import networkx as nx

In [2]:
FolderName=os.path.dirname(os.path.realpath("__file__"))
d_obj= DSS(r""+ FolderName+ "\ieee37.dss")  #creating a DSS object instance

In [3]:
d_obj.compile_ckt_dss() #compiling the circuit
d_obj.solve_snapshot_dss(1.0) #solving snapshot power flow

In [4]:
edges_dictlist=[]
G_original=nx.Graph()
for e in d_obj.get_AllPDElements():
    if ((e.split('.')[0]=='Line') or (e.split('.')[0]=='Transformer')): #only if its a line or transformer used in graph(capacitors avoided)
        branch_obj=Branch(d_obj,e) #creating  a branch object instance with full name
        sr_node=branch_obj.bus_fr.split('.')[0] #extracting source bus of branch
        tar_node=branch_obj.bus_to.split('.')[0] #extracting target bus of branch
        name=e.split('.')[1] #getting the name of the element(label or id)
        el_type=e.split('.')[0] #getting the type of the element (line or transformer)
        numphases=branch_obj.nphases #extracting number of phases of branch
        Inorm=branch_obj.Cap #extracting the baseflow current flow in branch
        Imax=branch_obj.MaxCap #extracting the maximum permissible current flow in branch
        Iresid=Imax-Inorm #residual current flow in branch
        if el_type== 'Line':
            R,X=d_obj.get_Line_RX(name)
        else:
            R,X=d_obj.get_Transformer_RX(name)
        edges_dictlist.append({"source":sr_node, "target":tar_node, "label":name, "device":el_type, "resistance":R, "reactance":X, "phases":numphases,"maxcap":Imax,"cap":Inorm,"residue":Iresid})
        G_original.add_edge(sr_node, tar_node, label=name, device=el_type, resistance=R, reactance=X, phases=numphases,maxcap=Imax,cap=Inorm,residue=Iresid)

In [5]:
buses=list(d_obj.get_AllBuses())
loads=d_obj.get_AllLoads()
buses.remove('sourcebus') #sourcebus not in 8500

In [6]:
for b in buses:
    P_sum=0
    Q_sum=0
    for j in range(len(loads)):
        if loads[j]["Bus"]==b:
            P_sum=P_sum+loads[j].get("Pload")
            Q_sum=Q_sum+loads[j].get("Qload")
    G_original.nodes[b]["Device"]="Bus"
    G_original.nodes[b]["ActiveLoad"]=P_sum
    G_original.nodes[b]["ReactiveLoad"]=Q_sum

In [7]:
G_original.edges(data=True)

EdgeDataView([('sourcebus', '799', {'label': 'subxf', 'device': 'Transformer', 'resistance': 9.216, 'reactance': 73.728, 'phases': 3, 'maxcap': 9.413319606352593, 'cap': 7.651991544342297, 'residue': 1.7613280620102953}), ('799', '799r', {'label': 'jumper', 'device': 'Line', 'resistance': 0.0003333333333333333, 'reactance': 0.0, 'phases': 1, 'maxcap': 600.0, 'cap': 0.0001791344731288204, 'residue': 599.9998208655269}), ('709', '775', {'label': 'xfm1', 'device': 'Transformer', 'resistance': 0.020735999999999997, 'reactance': 0.834048, 'phases': 3, 'maxcap': 90.21097956087903, 'cap': 3.9196186284099354e-05, 'residue': 90.21094036469275}), ('709', '731', {'label': 'l16', 'device': 'Line', 'resistance': 0.0930568182, 'reactance': 0.053064394000000015, 'phases': 3, 'maxcap': 600.0, 'cap': 13.101496206060945, 'residue': 586.8985037939391}), ('709', '708', {'label': 'l17', 'device': 'Line', 'resistance': 0.04963030303999999, 'reactance': 0.02830101013333334, 'phases': 3, 'maxcap': 600.0, 'cap

In [8]:
nx.readwrite.gml.write_gml(G_original,"37busEx.gml")